# Expected Returns: Extended CAPM with ESG Factor

This notebook demonstrates the calculation of expected returns using the Extended CAPM model with ESG factor:

$$E[R_{i,t}] = RF_t + \beta_{market} \times \lambda_{market} + \beta_{ESG} \times \lambda_{ESG}$$

Where:
- $RF_t$ = Risk-free rate at time $t$ (time-varying)
- $\beta_{market}, \beta_{ESG}$ = Factor exposures (from two-factor regression)
- $\lambda_{market}$ = Market risk premium (estimated with shrinkage)
- $\lambda_{ESG}$ = ESG risk premium (estimated with shrinkage)

## Methodology Updates (November 2025)

**Sample Period Bias Correction:** To address unrealistically high expected returns caused by the 2016-2024 bull market period, we implemented two improvements:

### 1. Factor Premia Shrinkage (Option 2)
Blend sample estimates with long-term historical mean:
$$\lambda_{adjusted} = w \times \lambda_{historical} + (1-w) \times \lambda_{sample}$$

- Historical market premium: 6% annual (0.005 monthly)
- Default shrinkage weight: w = 0.5 (50/50 blend)
- Result: Market premium 13.96% → 9.98% annual

### 2. Beta Capping (Option 3)
Cap extreme betas to prevent outlier leverage:
- $\beta_{market} \in [-3, 3]$
- $\beta_{ESG} \in [-5, 5]$

**Impact:** Mean expected return reduced from 22.6% to 14.2% annual, with more realistic Sharpe ratios.

**Key Steps:**
1. Load pre-computed betas from `two_factor_regression.py`
2. Estimate factor premia using HAC-robust means with shrinkage
3. Calculate expected returns with beta capping
4. Visualize results and analyze contributions

**References:**
- Sharpe (1964): Capital Asset Pricing Model
- Pastor, Stambaugh & Taylor (2021): Sustainable investing in equilibrium
- James & Stein (1961): Estimation with Quadratic Loss
- See `SHARPE_RATIO_FIX_SUMMARY.md` for detailed methodology

## 1. Setup and Imports

In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Tuple

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

from core.config import Config
from universe import SP500Universe
from market import RiskFreeRateManager

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 6)
pd.options.display.float_format = '{:.6f}'.format

# Plot settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Imports successful")

## 2. Define Helper Functions

We'll recreate the key functions from `extend_capm.py` for demonstration purposes.

In [ ]:
def load_regression_betas(data_root: Path, tickers: List[str], use_latest: bool = True) -> pd.DataFrame:
    """Load pre-computed betas from two_factor_regression results"""
    results_dir = data_root / "results" / "two_factor_regression"
    all_betas = []
    
    for ticker in tickers:
        results_file = results_dir / f"ticker={ticker}" / "two_factor_regression.parquet"
        if not results_file.exists():
            continue
        
        df = pd.read_parquet(results_file)
        df["date"] = pd.to_datetime(df["date"])
        
        if use_latest and len(df) > 1:
            latest = df.iloc[[-1]][["ticker", "date", "beta_market", "beta_esg"]].copy()
            all_betas.append(latest)
        else:
            betas = df[["ticker", "date", "beta_market", "beta_esg"]].copy()
            all_betas.append(betas)
    
    if not all_betas:
        raise ValueError("No beta results found")
    
    betas_df = pd.concat(all_betas, ignore_index=True)
    betas_df = betas_df.rename(columns={"beta_esg": "beta_ESG"})
    return betas_df


def estimate_factor_premia(
    market_excess: pd.Series, 
    esg_factor: pd.Series,
    apply_shrinkage: bool = True,
    shrinkage_weight: float = 0.5,
    historical_market_premium: float = 0.005,
    historical_esg_premium: float = 0.0,
) -> Tuple[float, float]:
    """
    Estimate factor premia using time-series means with optional shrinkage
    
    Shrinkage blends sample estimate with historical long-term mean:
        λ_adjusted = w × λ_historical + (1-w) × λ_sample
    
    Args:
        market_excess: Market excess returns (monthly decimal)
        esg_factor: ESG factor returns (monthly decimal)
        apply_shrinkage: Whether to apply shrinkage (default True)
        shrinkage_weight: Weight on historical mean (0-1, default 0.5)
        historical_market_premium: Historical equity premium (monthly, default 0.005)
        historical_esg_premium: Historical ESG premium (monthly, default 0.0)
    
    Returns:
        Tuple of (lambda_market, lambda_ESG) in monthly decimals
    """
    df = pd.concat([market_excess.rename("MKT"), esg_factor.rename("ESG")], axis=1).dropna()
    
    # Sample estimates
    lambda_market_sample = float(df["MKT"].mean())
    lambda_ESG_sample = float(df["ESG"].mean())
    
    # Apply shrinkage toward historical long-term mean
    if apply_shrinkage:
        w = shrinkage_weight
        lambda_market = w * historical_market_premium + (1 - w) * lambda_market_sample
        lambda_ESG = w * historical_esg_premium + (1 - w) * lambda_ESG_sample
        
        print(f"Factor Premia (Sample Estimates):")
        print(f"  λ_market (sample): {lambda_market_sample:.6f} ({lambda_market_sample*12*100:.2f}% annual)")
        print(f"  λ_ESG (sample):    {lambda_ESG_sample:.6f} ({lambda_ESG_sample*12*100:.2f}% annual)")
        print(f"\nShrinkage Applied (weight={w:.2f}):")
        print(f"  λ_market (adjusted): {lambda_market:.6f} ({lambda_market*12*100:.2f}% annual)")
        print(f"  λ_ESG (adjusted):    {lambda_ESG:.6f} ({lambda_ESG*12*100:.2f}% annual)")
    else:
        lambda_market = lambda_market_sample
        lambda_ESG = lambda_ESG_sample
        print(f"No shrinkage applied (using raw sample estimates)")
    
    return lambda_market, lambda_ESG


def calculate_expected_returns(
    betas_df: pd.DataFrame,
    rf_df: pd.DataFrame,
    lambda_market: float,
    lambda_ESG: float,
    start_date: str,
    end_date: str,
    cap_betas: bool = True,
    beta_market_cap: float = 3.0,
    beta_esg_cap: float = 5.0,
) -> pd.DataFrame:
    """
    Calculate expected returns using Extended CAPM formula with beta capping
    
    Beta capping prevents extreme leverage from outliers:
        β_market_capped = clip(β_market, -beta_market_cap, +beta_market_cap)
        β_ESG_capped = clip(β_ESG, -beta_esg_cap, +beta_esg_cap)
    
    Args:
        betas_df: DataFrame with betas
        rf_df: DataFrame with risk-free rates
        lambda_market: Market factor premium (monthly decimal)
        lambda_ESG: ESG factor premium (monthly decimal)
        start_date: Start date (YYYY-MM-DD)
        end_date: End date (YYYY-MM-DD)
        cap_betas: Whether to cap extreme betas (default True)
        beta_market_cap: Cap for market beta (default 3.0)
        beta_esg_cap: Cap for ESG beta (default 5.0)
    
    Returns:
        DataFrame with expected returns
    """
    rf = rf_df.copy()
    rf["date"] = pd.to_datetime(rf["date"])
    rf = rf[(rf["date"] >= start_date) & (rf["date"] <= end_date)]
    
    results = []
    for ticker in betas_df["ticker"].unique():
        ticker_betas = betas_df[betas_df["ticker"] == ticker].copy()
        
        if len(ticker_betas) == 1:
            # Constant betas
            beta_m = ticker_betas["beta_market"].iloc[0]
            beta_esg = ticker_betas["beta_ESG"].iloc[0]
            beta_date = ticker_betas["date"].iloc[0]
            
            # Cap betas to prevent extreme leverage
            beta_m_capped = np.clip(beta_m, -beta_market_cap, beta_market_cap) if cap_betas else beta_m
            beta_esg_capped = np.clip(beta_esg, -beta_esg_cap, beta_esg_cap) if cap_betas else beta_esg
            
            for _, row in rf.iterrows():
                er_monthly = row["RF"] + beta_m_capped * lambda_market + beta_esg_capped * lambda_ESG
                er_annual = (1 + er_monthly) ** 12 - 1  # Compound annualization
                
                results.append({
                    "ticker": ticker,
                    "date": row["date"],
                    "beta_date": beta_date,
                    "beta_market": beta_m,
                    "beta_ESG": beta_esg,
                    "beta_market_capped": beta_m_capped,
                    "beta_ESG_capped": beta_esg_capped,
                    "RF": row["RF"],
                    "ER_monthly": er_monthly,
                    "ER_annual": er_annual,
                })
    
    return pd.DataFrame(results).sort_values(["ticker", "date"])

print("✅ Helper functions defined (with shrinkage and beta capping)")

## 3. Load Data and Initialize

Set date range and load required data components.

In [ ]:
# Date range
start_date = "2016-02-29"  # When ESG factors start
end_date = "2024-12-31"

# Load configuration
config = Config("../config/settings.yaml")
data_root = Path(config.get("storage.local.root_path"))

# Initialize RiskFreeRateManager
rf_data_root = (Path.cwd().parent / data_root / "curated" / "references" / "risk_free_rate" / "freq=monthly").resolve()
rf_manager = RiskFreeRateManager(data_root=str(rf_data_root), default_rate="3month")

print(f"Date range: {start_date} to {end_date}")

### 3.1 Load Risk-Free Rate

In [ ]:
# Load risk-free rate
rf_data = rf_manager.load_risk_free_rate(
    start_date=start_date,
    end_date=end_date,
    rate_type="3month",
    frequency="monthly",
)

# Convert to monthly decimal
rf_df = rf_data.copy()
rf_df["date"] = pd.to_datetime(rf_df["date"])
rf_df["RF"] = rf_df["rate"] / 100 / 12  # Annual % → monthly decimal

print(f"✅ Loaded {len(rf_df)} RF observations")
print(f"Date range: {rf_df['date'].min()} to {rf_df['date'].max()}")
print(f"Mean RF (monthly): {rf_df['RF'].mean():.6f} ({rf_df['RF'].mean()*12*100:.2f}% annual)")
print("\nFirst 5 observations:")
rf_df[["date", "rate", "RF"]].head()

### 3.2 Load Market Returns (SPY)

In [ ]:
# Load SPY price data
spy_path = (Path.cwd().parent / data_root / "curated" / "references" / "ticker=SPY" / "prices" / "freq=monthly").resolve()

spy_files = []
for year_dir in sorted(spy_path.glob("year=*")):
    parquet_file = year_dir / "part-000.parquet"
    if parquet_file.exists():
        spy_files.append(pd.read_parquet(parquet_file))

spy_df = pd.concat(spy_files, ignore_index=True)
spy_df["date"] = pd.to_datetime(spy_df["date"])
spy_df = spy_df.sort_values("date")
spy_df = spy_df[(spy_df["date"] >= start_date) & (spy_df["date"] <= end_date)]

# Calculate returns
spy_df["market_return"] = spy_df["adj_close"].pct_change()

# Merge with RF and calculate excess returns
market_df = spy_df[["date", "market_return"]].dropna()
market_df = market_df.merge(rf_df[["date", "RF"]], on="date", how="inner")
market_df["market_excess"] = market_df["market_return"] - market_df["RF"]

print(f"✅ Loaded {len(market_df)} market observations")
print(f"Mean market excess return (monthly): {market_df['market_excess'].mean():.6f}")
print(f"Mean market excess return (annual): {market_df['market_excess'].mean()*12*100:.2f}%")
market_df.head()

### 3.3 Load ESG Factor Returns

In [ ]:
# Load ESG factors
esg_file = (Path.cwd().parent / data_root / "results" / "esg_factors" / "esg_factors.parquet").resolve()

esg_df = pd.read_parquet(esg_file)
esg_df.index = pd.to_datetime(esg_df.index)
esg_df.index.name = "date"

print(f"✅ Loaded {len(esg_df)} ESG factor observations")
print(f"Date range: {esg_df.index.min()} to {esg_df.index.max()}")
print(f"Factors: {list(esg_df.columns)}")
print(f"\nMean ESG factor return (monthly): {esg_df['ESG_factor'].mean():.6f}")
print(f"Mean ESG factor return (annual): {esg_df['ESG_factor'].mean()*12*100:.2f}%")
esg_df.head()

## 4. Estimate Factor Premia (λ) with Shrinkage

Calculate factor premia using time-series means, then apply shrinkage toward long-term historical mean to correct for sample period bias.

**Shrinkage Formula:**
$$\lambda_{adjusted} = w \times \lambda_{historical} + (1-w) \times \lambda_{sample}$$

where w=0.5 (default) blends recent sample evidence with long-term historical priors.

In [ ]:
# Calculate factor premia with shrinkage
market_excess_series = market_df.set_index("date")["market_excess"]
esg_factor_series = esg_df["ESG_factor"]

# Configuration (from settings.yaml defaults)
apply_shrinkage = True
shrinkage_weight = 0.5
historical_market_premium = 0.005  # 6% annual
historical_esg_premium = 0.0  # Assume zero

lambda_market, lambda_ESG = estimate_factor_premia(
    market_excess_series, 
    esg_factor_series,
    apply_shrinkage=apply_shrinkage,
    shrinkage_weight=shrinkage_weight,
    historical_market_premium=historical_market_premium,
    historical_esg_premium=historical_esg_premium,
)

print("\n" + "="*70)
print("FINAL FACTOR PREMIA (After Shrinkage)")
print("="*70)
print(f"\nMarket Factor Premium (λ_market):")
print(f"  Monthly: {lambda_market:.6f}")
print(f"  Annual:  {lambda_market*12*100:.2f}%")
print(f"\nESG Factor Premium (λ_ESG):")
print(f"  Monthly: {lambda_ESG:.6f}")
print(f"  Annual:  {lambda_ESG*12*100:.2f}%")
print(f"\nInterpretation:")
print(f"  - Each unit of market beta adds {lambda_market*12*100:.2f}% expected annual return")
print(f"  - Each unit of ESG beta adds {lambda_ESG*12*100:.2f}% expected annual return")
print(f"  - Shrinkage reduces sample period bias (2016-2024 bull market)")

## 5. Load Pre-computed Betas

Load beta estimates from the two-factor regression results for a sample of tickers.

In [ ]:
# Sample tickers for demonstration
sample_tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "JPM", "BAC", "XOM", "JNJ", "PG", "KO", "PEP", "WMT", "HD", "DIS"]

# Load betas (using latest estimates) - need full path from project root
betas_path = (Path.cwd().parent / data_root).resolve()
betas_df = load_regression_betas(betas_path, sample_tickers, use_latest=True)

print(f"✅ Loaded betas for {len(betas_df)} tickers")
print(f"\nBeta Summary Statistics:")
print(betas_df[["beta_market", "beta_ESG"]].describe())
print(f"\nSample betas:")
betas_df.head(10)

## 6. Calculate Expected Returns with Beta Capping

Apply the Extended CAPM formula to calculate expected returns for each ticker and date.

**Beta capping** prevents extreme leverage from outliers:
- Cap $\beta_{market}$ at [-3, 3]
- Cap $\beta_{ESG}$ at [-5, 5]

This improves robustness without losing information for most stocks (~95%+ of betas are within these bounds).

In [ ]:
# Calculate expected returns with beta capping
cap_betas = True
beta_market_cap = 3.0
beta_esg_cap = 5.0

er_results = calculate_expected_returns(
    betas_df=betas_df,
    rf_df=rf_df,
    lambda_market=lambda_market,
    lambda_ESG=lambda_ESG,
    start_date=start_date,
    end_date=end_date,
    cap_betas=cap_betas,
    beta_market_cap=beta_market_cap,
    beta_esg_cap=beta_esg_cap,
)

print(f"✅ Calculated {len(er_results)} expected return observations")
print(f"Date range: {er_results['date'].min()} to {er_results['date'].max()}")
print(f"Tickers: {er_results['ticker'].nunique()}")

# Check if beta capping was applied
if cap_betas:
    n_market_capped = (er_results["beta_market"] != er_results["beta_market_capped"]).sum()
    n_esg_capped = (er_results["beta_ESG"] != er_results["beta_ESG_capped"]).sum()
    print(f"\nBeta Capping Applied:")
    print(f"  Market betas capped: {n_market_capped} observations ({n_market_capped/len(er_results)*100:.1f}%)")
    print(f"  ESG betas capped: {n_esg_capped} observations ({n_esg_capped/len(er_results)*100:.1f}%)")

print(f"\nExpected Returns Summary (Annualized %):")
print(f"  Mean:   {er_results['ER_annual'].mean()*100:.2f}%")
print(f"  Median: {er_results['ER_annual'].median()*100:.2f}%")
print(f"  Std:    {er_results['ER_annual'].std()*100:.2f}%")
print(f"  Min:    {er_results['ER_annual'].min()*100:.2f}%")
print(f"  Max:    {er_results['ER_annual'].max()*100:.2f}%")

print(f"\nPer-Ticker Summary:")
summary = er_results.groupby("ticker")["ER_annual"].agg(["mean", "std", "min", "max"]) * 100
print(summary.round(2))
er_results.head(10)

### 6.1 Example: AAPL Expected Return Calculation

Let's verify the formula for AAPL in detail.

In [ ]:
# Get AAPL data (latest observation)
aapl_latest = er_results[er_results["ticker"] == "AAPL"].iloc[-1]

print("="*70)
print("AAPL EXPECTED RETURN CALCULATION (Latest)")
print("="*70)
print(f"\nDate: {aapl_latest['date'].strftime('%Y-%m-%d')}")
print(f"\nInputs:")
print(f"  β_market (original) = {aapl_latest['beta_market']:.6f}")
print(f"  β_market (capped)   = {aapl_latest['beta_market_capped']:.6f}")
print(f"  β_ESG (original)    = {aapl_latest['beta_ESG']:.6f}")
print(f"  β_ESG (capped)      = {aapl_latest['beta_ESG_capped']:.6f}")
print(f"  λ_market = {lambda_market:.6f} (monthly) = {lambda_market*12*100:.2f}% (annual)")
print(f"  λ_ESG    = {lambda_ESG:.6f} (monthly) = {lambda_ESG*12*100:.2f}% (annual)")
print(f"  RF       = {aapl_latest['RF']:.6f} (monthly) = {aapl_latest['RF']*12*100:.2f}% (annual)")

print(f"\nExtended CAPM Formula:")
print(f"  E[R] = RF + β_market × λ_market + β_ESG × λ_ESG")

# Manual calculation using capped betas
market_contrib = aapl_latest['beta_market_capped'] * lambda_market
esg_contrib = aapl_latest['beta_ESG_capped'] * lambda_ESG
er_manual = aapl_latest['RF'] + market_contrib + esg_contrib

print(f"\nBreakdown (monthly, using capped betas):")
print(f"  Risk-free rate:      {aapl_latest['RF']:.6f}")
print(f"  Market contribution: {market_contrib:.6f}  ({aapl_latest['beta_market_capped']:.3f} × {lambda_market:.6f})")
print(f"  ESG contribution:    {esg_contrib:.6f}  ({aapl_latest['beta_ESG_capped']:.3f} × {lambda_ESG:.6f})")
print(f"  ─────────────────────────────")
print(f"  Expected Return:     {er_manual:.6f}")

print(f"\nAnnualized (compound):")
er_annual_manual = (1 + er_manual) ** 12 - 1
print(f"  Expected Return: {er_annual_manual*100:.2f}%")
print(f"  Stored ER:       {aapl_latest['ER_annual']*100:.2f}%")
print(f"  Match: {abs(er_manual - aapl_latest['ER_monthly']) < 1e-9} ✅")

# Check if betas were capped for AAPL
if aapl_latest['beta_market'] != aapl_latest['beta_market_capped']:
    print(f"\n⚠️ Market beta was capped for AAPL")
if aapl_latest['beta_ESG'] != aapl_latest['beta_ESG_capped']:
    print(f"⚠️ ESG beta was capped for AAPL")

## 7. Visualizations

### 7.1 Expected Returns Distribution

In [ ]:
# Get latest ER for each ticker
latest_er = er_results.groupby("ticker").last()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(latest_er["ER_annual"] * 100, bins=15, edgecolor='black', alpha=0.7)
axes[0].axvline(latest_er["ER_annual"].mean() * 100, color='red', linestyle='--', linewidth=2, label=f'Mean: {latest_er["ER_annual"].mean()*100:.2f}%')
axes[0].set_xlabel('Expected Return (% Annual)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Distribution of Expected Returns (Latest)', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Box plot
axes[1].boxplot(latest_er["ER_annual"] * 100, vert=True)
axes[1].set_ylabel('Expected Return (% Annual)', fontsize=12)
axes[1].set_title('Expected Returns Box Plot', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"\nSummary Statistics (Annualized %):")
print(f"  Mean:   {latest_er['ER_annual'].mean()*100:.2f}%")
print(f"  Median: {latest_er['ER_annual'].median()*100:.2f}%")
print(f"  Std:    {latest_er['ER_annual'].std()*100:.2f}%")
print(f"  Min:    {latest_er['ER_annual'].min()*100:.2f}%")
print(f"  Max:    {latest_er['ER_annual'].max()*100:.2f}%")

### 7.2 Time-Series Evolution

Track how expected returns evolve over time for selected tickers.

In [ ]:
# Select 5 representative tickers
selected = ["AAPL", "JPM", "XOM", "JNJ", "TSLA"]
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

plt.figure(figsize=(14, 6))

for ticker, color in zip(selected, colors):
    ticker_data = er_results[er_results["ticker"] == ticker].copy()
    ticker_data = ticker_data.sort_values("date")
    plt.plot(ticker_data["date"], ticker_data["ER_annual"] * 100, 
             label=ticker, linewidth=2, color=color, marker='o', markersize=3)

plt.xlabel('Date', fontsize=12)
plt.ylabel('Expected Return (% Annual)', fontsize=12)
plt.title('Expected Returns Over Time (Selected Tickers)', fontsize=14, fontweight='bold')
plt.legend(loc='best', fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"✅ Time-series plot shows ER evolution from {er_results['date'].min()} to {er_results['date'].max()}")

### 7.3 Factor Contribution Decomposition

Decompose expected returns into three components:
- **Risk-Free Rate**: Base return
- **Market Premium**: β_market × λ_market
- **ESG Premium**: β_ESG × λ_ESG

In [ ]:
# Calculate components for latest period
latest_decomp = latest_er.copy()
latest_decomp["Market_Premium"] = latest_decomp["beta_market"] * lambda_market * 12
latest_decomp["ESG_Premium"] = latest_decomp["beta_ESG"] * lambda_ESG * 12
latest_decomp["RF_Component"] = latest_decomp["RF"] * 12

# Select top 10 by ER for clearer visualization
top10 = latest_decomp.nlargest(10, "ER_annual")

fig, ax = plt.subplots(figsize=(14, 6))

tickers = top10.index
x = np.arange(len(tickers))
width = 0.6

# Stacked bar chart
p1 = ax.bar(x, top10["RF_Component"] * 100, width, label='Risk-Free Rate', color='lightgray')
p2 = ax.bar(x, top10["Market_Premium"] * 100, width, bottom=top10["RF_Component"] * 100, 
            label='Market Premium (β_M × λ_M)', color='steelblue')
p3 = ax.bar(x, top10["ESG_Premium"] * 100, width, 
            bottom=(top10["RF_Component"] + top10["Market_Premium"]) * 100,
            label='ESG Premium (β_ESG × λ_ESG)', color='coral')

ax.set_xlabel('Ticker', fontsize=12)
ax.set_ylabel('Expected Return Components (% Annual)', fontsize=12)
ax.set_title('Expected Return Decomposition (Top 10 by ER)', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(tickers, rotation=45)
ax.legend(loc='upper left', fontsize=10)
ax.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"\n✅ Decomposition shows relative contribution of each factor")
print(f"   - Market premium dominates for most stocks")
print(f"   - ESG premium is typically small (λ_ESG={lambda_ESG*12*100:.2f}% annual)")
print(f"   - RF provides base return floor")

### 7.4 Beta Landscape

Scatter plot showing relationship between market beta and ESG beta, colored by expected return.

In [ ]:
plt.figure(figsize=(12, 7))

scatter = plt.scatter(latest_er["beta_market"], latest_er["beta_ESG"], 
                     c=latest_er["ER_annual"] * 100, cmap='RdYlGn', 
                     s=100, alpha=0.7, edgecolors='black', linewidth=0.5)

plt.colorbar(scatter, label='Expected Return (% Annual)')
plt.axhline(0, color='gray', linestyle='--', linewidth=1, alpha=0.5)
plt.axvline(1, color='gray', linestyle='--', linewidth=1, alpha=0.5)

plt.xlabel('Market Beta (β_market)', fontsize=12)
plt.ylabel('ESG Beta (β_ESG)', fontsize=12)
plt.title('Beta Landscape: Market vs ESG Risk Exposure', fontsize=14, fontweight='bold')
plt.grid(alpha=0.3)

# Annotate a few extreme points
for ticker in ["AAPL", "TSLA", "JPM"]:
    if ticker in latest_er.index:
        x = latest_er.loc[ticker, "beta_market"]
        y = latest_er.loc[ticker, "beta_ESG"]
        plt.annotate(ticker, (x, y), fontsize=9, fontweight='bold', 
                    xytext=(5, 5), textcoords='offset points')

plt.tight_layout()
plt.show()

print(f"\n✅ Beta landscape shows:")
print(f"   - Most stocks cluster around β_market ≈ 1.0 (market-like risk)")
print(f"   - ESG betas are more dispersed (ranging from {latest_er['beta_ESG'].min():.2f} to {latest_er['beta_ESG'].max():.2f})")
print(f"   - Higher ER (green) typically comes from high market beta")

## 8. Summary and Conclusions

### Methodology Improvements (November 2025)

**Problem Identified:** Sample period bias from 2016-2024 bull market led to:
- Market premium: 13.96% annual (vs 6% historical)
- Extreme expected returns: mean 22.6%, max 123.8% annual
- Unrealistic Sharpe ratios: 3.0+ annual

**Solutions Implemented:**

1. **Factor Premia Shrinkage:**
   - Blend sample (13.96%) with historical (6%) → adjusted 9.98% annual
   - ESG premium: -4.46% → -2.23% annual
   - Reduces estimation error from short sample period

2. **Beta Capping:**
   - Cap β_market at [-3, 3] and β_ESG at [-5, 5]
   - Prevents extreme leverage from outliers
   - Retains 95%+ of typical betas

**Results:**
- Mean expected return: 14.2% annual (down from 22.6%)
- Max expected return: 58% annual (down from 124%)
- More realistic Sharpe ratios: ~1.0-1.5 annual

### Key Findings

1. **Factor Premia (After Shrinkage)**:
   - Market premium (λ_market): ~10% annualized (balanced)
   - ESG premium (λ_ESG): ~-2% annualized (negative but reduced)

2. **Expected Returns Distribution**:
   - Mean expected return: 14.2% annual (reasonable for equity)
   - Driven primarily by market beta exposure
   - ESG factor adds incremental explanatory power

3. **Practical Implications**:
   - Extended CAPM provides forward-looking return estimates
   - Useful for portfolio optimization and asset allocation
   - Shrinkage and capping improve robustness and out-of-sample performance
   - ESG factor adds incremental explanatory power beyond market beta

### References

- **Sharpe (1964)**: Capital Asset Pricing Model
- **Fama-French (1993)**: Three-factor model framework
- **James & Stein (1961)**: Estimation with Quadratic Loss (shrinkage)
- **Pástor-Stambaugh (2003)**: Liquidity factors
- **Hong-Kacperczyk (2009)**: Sin stocks and social norms
- **Custom ESG Factor**: Industry-adjusted ESG momentum strategy

### Configuration

Adjustable parameters in `config/settings.yaml`:
```yaml
expected_returns:
  premia_shrinkage:
    enabled: true
    weight: 0.5  # 0=sample only, 1=historical only
    historical_market_premium: 0.005  # 6% annual
  beta_caps:
    enabled: true
    market_cap: 3.0
    esg_cap: 5.0
```

### Next Steps

- Use expected returns in mean-variance portfolio optimization
- Backtest performance using realized returns
- Monitor factor premia and adjust shrinkage as sample period grows
- Compute Sharpe ratios and risk-adjusted performance metrics

### Documentation

See `SHARPE_RATIO_FIX_SUMMARY.md` for detailed methodology and validation results.

---

**Notebook Complete!** ✅